## Setup a classification experiment

In [1]:
import pickle
import numpy as np

In [2]:
m1 = [0, +1 , -1 , 5]
w1= [1,2,3]
edges1 =[1,50,100]
print(f"Model 1 Vaues: {m1} \nModel 1 Edges: {edges1}")

m2 = [0,+2, -2, +2 , +3 , 9]
w2 =[10,20,30,40,50]
edges2 =[20,33,66,90 , 120]
print(f"Model 2 Vaues: {m1} \nModel 2 Edges: {edges1}")

merged_edges = sorted(set().union(*[set(edges1) , set(edges2)]))
print("\nMerged Edges = \n" , merged_edges)

models =[ m1 , m2]
weights= [w1,w2]

ne = [ np.searchsorted( edges1,  merged_edges + [np.inf] ) , np.searchsorted(edges2,  merged_edges + [np.inf])]
print('Edge Index= ')
for v in ne: print(v)


bv =[]
wv=[]
print('New Model Values: ')
for idx, m in enumerate(models):
    mvalues = m[1:]    
    new_val = [ mvalues[x-1] if x > 0 and x <=len(mvalues) else 0 for x in ne[idx][1:] ]
    print(new_val)
    bv.append( new_val)
    wv.append([ weights[idx][x-1] if x > 0 and x <=len(mvalues) else 0 for x in ne[idx][1:] ])

mbv = np.average(bv, axis=0, weights= wv)# np.nanmean(bv, axis=0)

print("Merged models values= \n" , mbv)

# Merged Model=  [1.   1.5 -0.5 -1.5  0.5  1.   4.   7. ]

Model 1 Vaues: [0, 1, -1, 5] 
Model 1 Edges: [1, 50, 100]
Model 2 Vaues: [0, 1, -1, 5] 
Model 2 Edges: [1, 50, 100]

Merged Edges = 
 [1, 20, 33, 50, 66, 90, 100, 120]
Edge Index= 
[0 1 1 1 2 2 2 3 3]
[0 0 1 2 2 3 4 4 5]
New Model Values: 
[1, 1, 1, -1, -1, -1, 5, 5]
[0, 2, -2, -2, 2, 3, 3, 9]
Merged models values= 
 [ 1.          1.90909091 -1.85714286 -1.90909091  1.8125      2.80952381
  3.13953488  8.77358491]


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [18]:
from interpret import show
from interpret.data import ClassHistogram

# hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
# show(hist)

## Train the Explainable Boosting Machine (EBM)

In [3]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

## Mergring Multiple Explainable Boosting Machines (EBM)

In [4]:
import numpy as np
from sklearn.model_selection import KFold
from interpret.glassbox import ExplainableBoostingClassifier

rng = np.random.RandomState(0)
cv = KFold(n_splits=3, shuffle=True, random_state=rng)
models =[]
seed =1
for train_index, test_index in cv.split(X):

    X_train, y_train = X.iloc[train_index], y.iloc[train_index]

    print( len(X_train) , len(y_train))
    
    ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
    ebm.fit(X_train, y_train)  

    models.append(ebm)
    
    seed +=10


21707 21707
21707 21707
21708 21708


In [7]:
%load_ext autoreload
%autoreload 2
from interpret.glassbox.ebm.utils import *
from interpret import show

merged_ebm = EBMUtils.merge_models(models=models)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
ebm_global = merged_ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7001/1424720715656/ -->

In [9]:
ebm0_global = models[0].explain_global(name='EBM')
show(ebm0_global)

<!-- http://127.0.0.1:7001/1425217374280/ -->

In [10]:
ebm1_global = models[1].explain_global(name='EBM')
show(ebm1_global)

<!-- http://127.0.0.1:7001/1425217375688/ -->

In [11]:
ebm2_global = models[2].explain_global(name='EBM')
show(ebm2_global)

<!-- http://127.0.0.1:7001/1425249590792/ -->

In [12]:
seed =1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
ebm1 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
print( len(X_train) , len(y_train))
ebm1.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
print( len(X_train) , len(y_train))
ebm2 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm2.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
print( len(X_train) , len(y_train))
ebm3 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm3.fit(X_train, y_train)  

26048 26048
26048 26048
26048 26048


ExplainableBoostingClassifier(feature_names=['Age', 'WorkClass', 'fnlwgt',
                                             'Education', 'EducationNum',
                                             'MaritalStatus', 'Occupation',
                                             'Relationship', 'Race', 'Gender',
                                             'CapitalGain', 'CapitalLoss',
                                             'HoursPerWeek', 'NativeCountry',
                                             'Age x Relationship',
                                             'Relationship x HoursPerWeek',
                                             'Relationship x CapitalLoss',
                                             'WorkClass x Race',
                                             'Age x HoursPerWeek',
                                             'EducationNum x Occupati...
                              feature_types=['continuous', 'categorical',
                                             '

In [17]:
from interpret.glassbox.ebm.utils import *

merged_ebm2 = EBMUtils.merge_models(models=[ebm1, ebm2 , ebm3])

ebm_global2 = merged_ebm.explain_global(name='EBM')
show(ebm_global2)

<!-- http://127.0.0.1:7001/1425268148808/ -->

In [16]:
# pickle.dump(ebm1,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm1.pickle", 'wb'))
# pickle.dump(ebm2,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm2.pickle", 'wb'))
# pickle.dump(ebm3,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm3.pickle", 'wb'))

# ebm1 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm1.pickle" ,'rb'))
# ebm2 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm2.pickle" ,'rb'))
# ebm3 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm3.pickle" ,'rb'))

In [15]:
# (ebm1.preprocessor_.col_bin_counts_[0])

## Global Explanations: What the model learned overall

In [14]:
ebm_global2 = merged_ebm.explain_global(name='EBM')
show(ebm_global2)

<!-- http://127.0.0.1:7001/1425266539656/ -->

## Local Explanations: How an individual prediction was made

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

## Evaluate EBM performance

In [ ]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

## Let's test out a few other Explainable Models

In [ ]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l1', solver='liblinear')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

## Compare performance using the Dashboard

In [ ]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

## Glassbox: All of our models have global and local explanations

In [ ]:
lr_global = lr.explain_global(name='Logistic Regression')
tree_global = tree.explain_global(name='Classification Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

## Dashboard: look at everything at once

In [ ]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([hist, lr_global, lr_perf, tree_global, tree_perf, ebm_global, ebm_perf], share_tables=True)